In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler


from statsmodels.stats.weightstats import _zconfint_generic, _tconfint_generic

from tqdm import tqdm_notebook
from IPython import display

import warnings
warnings.filterwarnings("ignore")

In [7]:
path = r'C:\Users\Public\Documents\HRS\Features new'
beginning_of_name_EL = '\Feature matrix EL no noise'
beginning_of_name_EL_OM = '\Feature matrix EL+OM no noise'
end_of_name = 'peaks.csv'
list_of_names = [' 3 ', ' 4 ', ' 5 ']

f1 = np.zeros((3,2)) # list of f1_scores, shape list_of_names*3
acc = np.zeros((3,2)) # list of accuracies, shape list_of_names*3
f1_left_confint = np.zeros((3,2))
f1_right_confint = np.zeros((3,2))

for i in range(len(list_of_names)):
    data = pd.read_csv(path + beginning_of_name_EL + list_of_names[i] + end_of_name, index_col=0)
    data.drop(['Label for OM and EL'], axis=1, inplace=True)
    data1 = pd.read_csv(path + beginning_of_name_EL_OM + list_of_names[i] + end_of_name, index_col=0)
    print(list_of_names[i] + end_of_name)
    
    X = data.drop('target', axis=1) #full of EL data ~ 900 sequencies
    y = data['target']

    X1 = data1.drop('target', axis=1) #full of EL+OM data ~ 200 sequencies
    y1 = data1['target']

    scaler = StandardScaler()
    
    X = scaler.fit_transform(X)
    X1 = scaler.fit_transform(X1)
     

    (X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size=0.3, stratify=y)

    (X1_train, X1_test, y1_train, y1_test) = train_test_split(X1, y1,\
                                                                  test_size=0.3, stratify=y1)
    
    X_resampled, y_resampled = SMOTE().fit_resample(X_train, y_train)
    print(X_resampled.shape, y_resampled.shape)
    X1_resampled, y1_resampled = SMOTE().fit_resample(X1_train, y1_train)
    print(X1_resampled.shape, y1_resampled.shape)

    
#     parameters = {'kernel':('linear', 'rbf'), 'C':[1, 2, 5, 7, 10]}


    clf = SVC(kernel='rbf', C=1.0, gamma='scale', class_weight='balanced')
    clf.fit(X_resampled, y_resampled)
    
    clf1 = SVC(kernel='rbf', C=1.0, gamma='scale', class_weight='balanced') 
    clf1.fit(X1_resampled, y1_resampled)
    
    y_pred = clf.predict(X_test)
    y1_pred = clf1.predict(X1_test)
    
    acc[i][0]= (accuracy_score(y_test, y_pred) * 100.0)
    acc[i][1]= (accuracy_score(y1_test, y1_pred) * 100.0)
    
    f1[i][0]= f1_score(y_test, y_pred)
    f1[i][1]= f1_score(y1_test, y1_pred)

    f1score = cross_val_score(clf, X_resampled, y_resampled, scoring='f1', cv=10)
    f1score1 = cross_val_score(clf, X1_resampled, y1_resampled, scoring='f1', cv=10)

    mean = f1score.mean()
    mean1 = f1score1.mean()
    
    print("SVC model EL f1: mean %.3f, std %.3f" % (f1score.mean(), f1score.std(ddof=1)))
    print("SVC model EL+OM f1: mean %.3f, std %.3f" % (f1score1.mean(), f1score1.std(ddof=1)))
    
    mean_std = f1score.std(ddof=1)/np.sqrt(len(f1score))
    mean_std1 = f1score1.std(ddof=1)/np.sqrt(len(f1score1))
    
    tconf = _tconfint_generic(mean, mean_std, len(f1score) - 1, 0.05, 'two-sided')
    tconf1 = _tconfint_generic(mean1, mean_std1, len(f1score1) - 1, 0.05, 'two-sided')

    print("SVC model EL mean f1 95%% confidence interval:", tconf)
    print("SVC model EL+OM mean f1 95%% confidence interval:", tconf1)
    
#     optimizer = GridSearchCV(clf, parameters, cv=10, scoring='f1')
#     optimizer.fit(X_train, y_train) 
#     print(optimizer.best_score_, optimizer.best_params_)
    
    

#     optimizer1 = GridSearchCV(clf, parameters, cv=10, scoring='f1')
#     optimizer1.fit(X1_train, y1_train) 
#     print(optimizer1.best_score_, optimizer1.best_params_)


#     y_pred = optimizer.best_estimator_.predict(X_test)
#     y_pred1 = optimizer1.best_estimator_.predict(X1_test)

#     f1[i][0]= f1_score(y_test, y_pred)
#     f1[i][1]= f1_score(y1_test, y_pred1)

    

 3 peaks.csv
(1098, 15) (1098,)
(296, 30) (296,)
SVC model EL f1: mean 0.875, std 0.024
SVC model EL+OM f1: mean 0.866, std 0.066
SVC model EL mean f1 95%% confidence interval: (0.8578810527940206, 0.8926666813308279)
SVC model EL+OM mean f1 95%% confidence interval: (0.8189150341940713, 0.9137961900243831)
 4 peaks.csv
(1098, 19) (1098,)
(296, 38) (296,)
SVC model EL f1: mean 0.900, std 0.019
SVC model EL+OM f1: mean 0.869, std 0.047
SVC model EL mean f1 95%% confidence interval: (0.8863208988849325, 0.9137029988701408)
SVC model EL+OM mean f1 95%% confidence interval: (0.8355864591953184, 0.903205259838336)
 5 peaks.csv
(1098, 23) (1098,)
(296, 46) (296,)
SVC model EL f1: mean 0.914, std 0.026
SVC model EL+OM f1: mean 0.884, std 0.044
SVC model EL mean f1 95%% confidence interval: (0.8953302120316042, 0.9323551439728859)
SVC model EL+OM mean f1 95%% confidence interval: (0.8525864662458317, 0.9152457220134652)


In [8]:
f1

array([[0.10810811, 0.61904762],
       [0.17857143, 0.48888889],
       [0.2       , 0.4       ]])

In [9]:
acc

array([[73.80952381, 79.48717949],
       [81.74603175, 70.51282051],
       [84.12698413, 69.23076923]])